In [ ]:
CLOUD_API_KEY = "YOU API KEY"

In [ ]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install pyspark==2.3 --no-cache | tail -n 1
!pip install numpy==1.15.4 --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [ ]:
from ibm_ai_openscale.utils import get_instance_guid


WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

In [ ]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

In [ ]:
subscription_uid = '809c000e-de6c-473a-9305-d18356704e2a'

In [ ]:
import json

from ibm_ai_openscale.supporting_classes import PayloadRecord

subscription = ai_client.data_mart.subscriptions.get(subscription_uid=subscription_uid)

In [ ]:
# from ibm_ai_openscale.engines import CustomMachineLearningAsset

# subscription = client.data_mart.subscriptions.add(CustomMachineLearningAsset("de5a31fe-f98b-4516-a303-e08b35800c09",
#                                                                             prediction_column="predicted_label",
#                                                                             probability_column="probability"))

In [ ]:
subscription.get_details()

In [ ]:
!rm bank_market_payload.csv
!wget https://raw.githubusercontent.com/ravichamarthy/custom-engine/master/bank_market_payload.csv

In [ ]:
import pandas as pd

df = pd.read_csv("bank_market_payload.csv")
df.head()

In [ ]:
len(df)

In [ ]:
label_col = "y"
feature_cols = list(df.columns)
feature_cols.remove(label_col)
categorical_cols = list(df[feature_cols].select_dtypes("object").columns)
print(feature_cols)
print(categorical_cols)


In [ ]:
input_values = df[feature_cols].values.tolist()
print(input_values[:10])

In [ ]:
import requests

# url = "http://localhost:5000/v1/deployments/6c2b3f6d-2c17-4b56-98ea-581a4981df6b/online"
url = "http://custom-serve-engine.mybluemix.net/v1/deployments/6c2b3f6d-2c17-4b56-98ea-581a4981df6b/online"

payload = {
    "fields": feature_cols,
    "values": input_values[:200]
}

from datetime import datetime, timedelta

start = datetime.utcnow()
response = requests.post(url, json=payload, auth=('admin', 'password'))
response_time = (datetime.utcnow() - start).total_seconds() * 1000
print(json.dumps(payload))
print(response.text)

In [ ]:
records = [PayloadRecord(request=payload, response=response.json(), response_time=int(response_time))]

subscription.payload_logging.store(records=records)

In [ ]:
time.sleep(10)
subscription.payload_logging.get_records_count()

In [ ]:
subscription.payload_logging.get_table_content()